In [2]:
import matplotlib.pyplot as plt
import json

## Overall line coverage
<img src="ss.png"/>

In [34]:
with open('json_outputs/mongod_main.gcov.json' ,'r') as f:
    data=json.load(f)
    data=data['files']
    print(data[0].keys())

dict_keys(['lines', 'functions', 'file'])


In [20]:
def gen_demangled_index(data):
    demangled_index={}
    for function in data['functions']:
        demangled_index[function['name']]=function['demangled_name']
    return demangled_index
demangled_index=(gen_demangled_index(data[0]))


In [21]:
data[0]['lines'][0]

{'branches': [],
 'count': 6,
 'line_number': 265,
 'unexecuted_block': False,
 'function_name': '_ZN5mongo12_GLOBAL__N_110logStartupEPNS_16OperationContextE'}

#### data[i]['lines'] has the data of every line of file i. 

In [36]:
def get_line_coverage(data):
    cnt=0
    for file in data:
        if file['file'].endswith('.cpp'):
            for line in file['lines']:
                if(line['count']>0):
                    cnt+=1
            return (cnt,len(file['lines']))
get_line_coverage(data)

(455, 728)

In [28]:

h_files=0
cpp_files=0
for file in data:
    if file['file'].endswith('.h') or file['file'].endswith('.hpp'):
        h_files+=1
    else:
        cpp_files+=1

print(h_files,cpp_files)

21 1


The number of cpp files in a gcov file is just 1 while the remaining files in the data are all header files. We will find line coverage only for the cpp files

In [38]:
def get_function_coverage(data):
    cnt=0
    for file in data:
        if file['file'].endswith('.cpp'):
            for function in file['functions']:
                if(function['execution_count']>0):
                    cnt+=1
            return (cnt,len(file['functions']))
get_function_coverage(data)


(19, 31)

#### To find the top 3 lengthiest unexecuted functions in the .cpp file


In [48]:
def get_longest_unused_functions(data):
    unex_func=[]
    for file in data:
        if file['file'].endswith('.cpp'):
            for function in file['functions']:
                if(function['execution_count']==0):
                    unex_func.append((function['demangled_name'],function['end_line']-function['start_line']))
            unex_func=sorted(unex_func,key=lambda x:-x[1])
            return unex_func[:3]
get_longest_unused_functions(data)

[('mongo::(anonymous namespace)::shutdownProcessByDBPathPidFile(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&)',
  54),
 ('mongo::(anonymous namespace)::makeReplicaSetNodeExecutor(mongo::ServiceContext*)',
  14),
 ('mongo::(anonymous namespace)::initializeCommandHooks(mongo::ServiceContext*)::MongodCommandInvocationHooks::onAfterAsyncRun(std::shared_ptr<mongo::RequestExecutionContext>, mongo::CommandInvocation*)',
  4)]